<a href="https://colab.research.google.com/github/khered20/UniManc_NADI2023_ArabicDialectToMSA_MT/blob/main/AraT5v2_DialecticalArabicToMSA_MT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install accelerate datasets sacrebleu sentencepiece evaluate rouge_score
!pip install --upgrade transformers==4.24.0


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=2c6881d2aed363ec43e5fd38d057061384e235634c96f1ef61ef6f4012dcdaf7
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [2]:
!wget https://raw.githubusercontent.com/UBC-NLP/araT5/main/examples/run_trainier_seq2seq_huggingface.py?token=AA4R7KKKCBPS5WX4BJSWNFTBG2OPK -O run_trainier_seq2seq_huggingface.py
!wget https://raw.githubusercontent.com/UBC-NLP/araT5/main/examples/eval_squad.py?token=AA4R7KOBBXSPCDXRRHJ3RW3BG2RF4 -O eval_squad.py


--2023-10-18 15:20:12--  https://raw.githubusercontent.com/UBC-NLP/araT5/main/examples/run_trainier_seq2seq_huggingface.py?token=AA4R7KKKCBPS5WX4BJSWNFTBG2OPK
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36637 (36K) [text/plain]
Saving to: ‘run_trainier_seq2seq_huggingface.py’

run_trainier_seq2se 100%[===================>]  35.78K  --.-KB/s    in 0.005s  

2023-10-18 15:20:13 (6.37 MB/s) - ‘run_trainier_seq2seq_huggingface.py’ saved [36637/36637]

--2023-10-18 15:20:13--  https://raw.githubusercontent.com/UBC-NLP/araT5/main/examples/eval_squad.py?token=AA4R7KOBBXSPCDXRRHJ3RW3BG2RF4
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubus

Training a joint model for all regional dialects (J-R).
<br> prepare a csv file for the NADI dev dataset where column "0" is the dialectical text and "1" is the corresponding MSA

In [11]:
!python run_trainier_seq2seq_huggingface.py \
        --learning_rate 5e-5 \
        --max_target_length 128 --max_source_length 128 \
        --per_device_train_batch_size 16 --per_device_eval_batch_size 16 \
        --model_name_or_path "UBC-NLP/AraT5v2-base-1024" \
        --output_dir "s2/AraT5_baseV2_100/regMADAR" --overwrite_output_dir \
        --save_steps 1000 \
        --eval_steps 1000 \
        --num_train_epochs 2 \
        --train_file "datasets/regMADARmsa.csv" \
        --validation_file "datasets/NADI_dev.csv" \
        --test_file "datasets/NADI_dev.csv" \
        --task "machine_translation" --text_column "0" --summary_column "1" \
        --load_best_model_at_end --metric_for_best_model "eval_bleu" --greater_is_better True --evaluation_strategy steps --logging_strategy epoch --predict_with_generate\
        --do_train --do_eval --do_predict

2023-10-18 15:45:04.676520: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-18 15:45:06.516811: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
last_checkpoint None
10/18/2023 15:45:11 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
10/18/2023 15:45:11 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None

Training an independent model for each regional dialect (I-R)
<br> Split the NADI-dev dataset to validated each model

In [12]:
# Train on Gulf and validate on Emirates_NADI-dev
!python run_trainier_seq2seq_huggingface.py \
        --learning_rate 5e-5 \
        --max_target_length 128 --max_source_length 128 \
        --per_device_train_batch_size 16 --per_device_eval_batch_size 16 \
        --model_name_or_path "UBC-NLP/AraT5v2-base-1024" \
        --output_dir "s2/AraT5_baseV2_100/emiMADAR" --overwrite_output_dir \
        --save_steps 1000 \
        --eval_steps 1000 \
        --num_train_epochs 2 \
        --train_file "datasets/glfMADARmsa.csv" \
        --validation_file "datasets/emiNADI_dev.csv" \
        --test_file "datasets/emiNADI_dev.csv" \
        --task "machine_translation" --text_column "0" --summary_column "1" \
        --load_best_model_at_end --metric_for_best_model "eval_bleu" --greater_is_better True --evaluation_strategy steps --logging_strategy epoch --predict_with_generate\
        --do_train --do_eval --do_predict

2023-10-18 16:32:04.069212: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-18 16:32:05.769617: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
last_checkpoint None
10/18/2023 16:32:09 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
10/18/2023 16:32:09 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None

In [13]:
# Train on Egyption and validate on Egyption_NADI-dev
!python run_trainier_seq2seq_huggingface.py \
        --learning_rate 5e-5 \
        --max_target_length 128 --max_source_length 128 \
        --per_device_train_batch_size 16 --per_device_eval_batch_size 16 \
        --model_name_or_path "UBC-NLP/AraT5v2-base-1024" \
        --output_dir "s2/AraT5_baseV2_100/egyMADAR" --overwrite_output_dir \
        --save_steps 1000 \
        --eval_steps 1000 \
        --num_train_epochs 2 \
        --train_file "datasets/egyMADARmsa.csv" \
        --validation_file "datasets/egyNADI_dev.csv" \
        --test_file "datasets/egyNADI_dev.csv" \
        --task "machine_translation" --text_column "0" --summary_column "1" \
        --load_best_model_at_end --metric_for_best_model "eval_bleu" --greater_is_better True --evaluation_strategy steps --logging_strategy epoch --predict_with_generate\
        --do_train --do_eval --do_predict

2023-10-18 16:52:50.988345: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-18 16:52:53.063251: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
last_checkpoint None
10/18/2023 16:52:57 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
10/18/2023 16:52:57 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None

In [ ]:
# Train on Levantine and validate on palestinian_NADI-dev
!python run_trainier_seq2seq_huggingface.py \
        --learning_rate 5e-5 \
        --max_target_length 128 --max_source_length 128 \
        --per_device_train_batch_size 16 --per_device_eval_batch_size 16 \
        --model_name_or_path "UBC-NLP/AraT5v2-base-1024" \
        --output_dir "s2/AraT5_baseV2_100/palMADAR" --overwrite_output_dir \
        --save_steps 1000 \
        --eval_steps 1000 \
        --num_train_epochs 2 \
        --train_file "datasets/levMADARmsa.csv" \
        --validation_file "datasets/palNADI_dev.csv" \
        --test_file "datasets/palNADI_dev.csv" \
        --task "machine_translation" --text_column "0" --summary_column "1" \
        --load_best_model_at_end --metric_for_best_model "eval_bleu" --greater_is_better True --evaluation_strategy steps --logging_strategy epoch --predict_with_generate\
        --do_train --do_eval --do_predict

2023-10-18 17:12:23.804947: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-18 17:12:25.447049: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
last_checkpoint None
10/18/2023 17:12:30 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
10/18/2023 17:12:30 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None

In [ ]:
# Train on Levantine and validate on Jordanian_NADI-dev
!python run_trainier_seq2seq_huggingface.py \
        --learning_rate 5e-5 \
        --max_target_length 128 --max_source_length 128 \
        --per_device_train_batch_size 16 --per_device_eval_batch_size 16 \
        --model_name_or_path "UBC-NLP/AraT5v2-base-1024" \
        --output_dir "s2/AraT5_baseV2_100/jorMADAR" --overwrite_output_dir \
        --save_steps 1000 \
        --eval_steps 1000 \
        --num_train_epochs 2 \
        --train_file "datasets/levMADARmsa.csv" \
        --validation_file "datasets/jorNADI_dev.csv" \
        --test_file "datasets/jorNADI_dev.csv" \
        --task "machine_translation" --text_column "0" --summary_column "1" \
        --load_best_model_at_end --metric_for_best_model "eval_bleu" --greater_is_better True --evaluation_strategy steps --logging_strategy epoch --predict_with_generate\
        --do_train --do_eval --do_predict